# Data Understanding

In [1]:
#Packages
import pandas as pd
import numpy as np
import sklearn

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [199]:
#Load data
train = pd.read_csv("sentiment_train.csv")
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB


,Sentence,Polarity
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [171]:
test = pd.read_csv("sentiment_test.csv")
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  600 non-null    object
 1   Polarity  600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ KB


,Sentence,Polarity
0,A good commentary of today's love and undoubte...,1
1,For people who are first timers in film making...,1
2,"It was very popular when I was in the cinema, ...",1
3,It's a feel-good film and that's how I felt wh...,1
4,It has northern humour and positive about the ...,1


In [172]:
# Is the target balanced?
count_col=train['Polarity']
pd.concat([count_col.value_counts(), 
                count_col.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage'))

,counts,percentage
0,1213,50.541667
1,1187,49.458333


The dataset is almost exactly balanced so we won't have to worry about any class balancing techniques.

# Data Preparation
## Train-Test Split

In [173]:
from sklearn.model_selection import train_test_split

X = train['Sentence']
y = train['Polarity']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Text Preprocessor

In [174]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import unidecode
import textstat
import string  

lemmer = WordNetLemmatizer()

# Simple preprocessor.
# Input is a single document, as a single string.
# Otuput should be a single document, as a single string.
def my_preprocess(doc):
    
    # Lowercase
    doc = doc.lower()
    
    # Replace all numbers/digits with the string NUM
    doc = re.sub(r'\b\d+\b', 'NUM', doc)
    
    # Lemmatize each word.
    doc = ' '.join([lemmer.lemmatize(w) for w in doc.split()])

    return doc

In [175]:
# These functions will calculate additional features on the document.
# They will be put into the Pipeline, called via the FunctionTransformer() function.
# Each one takes an entier corpus (as a list of documents), and should return
# an array of feature values (one for each document in the corpus).

def doc_length(corpus):
    return np.array([len(doc) for doc in corpus]).reshape(-1, 1)

def lexicon_count(corpus):
    return np.array([textstat.lexicon_count(doc) for doc in corpus]).reshape(-1, 1)

def _get_punc(doc):
    return len([a for a in doc if a in string.punctuation])

def punc_count(corpus):
    return np.array([_get_punc(doc) for doc in corpus]).reshape(-1, 1)

def _get_caps(doc):
    return sum([1 for a in doc if a.isupper()])

def capital_count(corpus):
    return np.array([_get_caps(doc) for doc in corpus]).reshape(-1, 1)

def num_exclamation_marks(corpus):
    return np.array([doc.count('!') for doc in corpus]).reshape(-1, 1)

def num_question_marks(corpus):
    return np.array([doc.count('?') for doc in corpus]).reshape(-1, 1)

def has_money(corpus):
    return np.array([bool(re.search("[\$£]|\bpence\b|\bdollar\b", doc.lower())) for doc in corpus]).reshape(-1, 1)

from textblob import TextBlob

def sent_polarity(corpus):
    return np.array([TextBlob(doc).sentiment.polarity for doc in corpus]).reshape(-1, 1)

# Modeling - Baseline Model
The baseline model includes TFIDF vectorization, topic modelling, and addtional features on the document

In [176]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF
from sklearn.neural_network import MLPClassifier

# Need to preprocess the stopwords, because scikit learn's TfidfVectorizer
# removes stopwords _after_ preprocessing
stop_words = [my_preprocess(word) for word in stop_words.ENGLISH_STOP_WORDS]

# This vectorizer will be used to create the BOW features
vectorizer = TfidfVectorizer(preprocessor=my_preprocess, 
                             max_features = 1000, 
                             ngram_range=[1,4],
                             stop_words=None,
                             strip_accents="unicode", 
                             lowercase=False, max_df=0.25, min_df=0.001, use_idf=True)

# This vectorizer will be used to preprocess the text before topic modeling.
# (I _could_ use the same vectorizer as above- but why limit myself?)
vectorizer2 = TfidfVectorizer(preprocessor=my_preprocess, 
                             max_features = 1000, 
                             ngram_range=[1,2],
                             stop_words=None,
                             strip_accents="unicode", 
                             lowercase=False, max_df=0.25, min_df=0.001, use_idf=True)

nmf = NMF(n_components=25, random_state=1, init='nndsvda', solver='mu', alpha=.1, l1_ratio=.5)

# We are using a Random Forest classifier
rf = RandomForestClassifier(criterion='entropy', random_state=223)

feature_processing =  FeatureUnion([ 
    ('bow', Pipeline([('cv', vectorizer), ])),
    ('topics', Pipeline([('cv', vectorizer2), ('nmf', nmf),])),
    ('length', FunctionTransformer(doc_length, validate=False)),
    ('words', FunctionTransformer(lexicon_count, validate=False)),
    ('punc_count', FunctionTransformer(punc_count, validate=False)),
    ('capital_count', FunctionTransformer(capital_count, validate=False)),  
    ('num_exclamation_marks', FunctionTransformer(num_exclamation_marks, validate=False)),  
    ('num_question_marks', FunctionTransformer(num_question_marks, validate=False)),  
    ('has_money', FunctionTransformer(has_money, validate=False)),
])

steps = [('features', feature_processing)]

pipe = Pipeline([('features', feature_processing), ('clf', rf)])

#Paramter grid for hyperparameter tuning
param_grid = {
    'features__bow__cv__preprocessor': [None, my_preprocess],
    'features__bow__cv__max_features': [200, 500, 1000],
    'features__bow__cv__use_idf': [True, False],
    'features__topics__cv__stop_words': [None],
    'features__topics__nmf__n_components': [25, 75],
    'clf__n_estimators': [100, 500],
}
    
#3-fold cross validation
search = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3, scoring='f1_micro', return_train_score=True, verbose=2)


In [177]:
search = search.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   29.1s
[Parallel(n_jobs=3)]: Done 144 out of 144 | elapsed:  4.2min finished


In [200]:
print("Best parameter (CV f1 score %0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV f1 score 0.739):
{'clf__n_estimators': 500, 'features__bow__cv__max_features': 1000, 'features__bow__cv__preprocessor': <function my_preprocess at 0x000001E3CB373E58>, 'features__bow__cv__use_idf': False, 'features__topics__cv__stop_words': None, 'features__topics__nmf__n_components': 75}


## Baseline Model - Evaluation on Test Set
Evaluating on the test set shows that the model works, and yielded F1 score of 0.79375.

In [179]:
# The pipeline with the best performance
pipeline = search.best_estimator_

# Get the feature processing pipeline, so I can use it later
feature_processing_obj = pipeline.named_steps['features']

In [180]:
# Print out the performance on the test set
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score

features_val = feature_processing_obj.transform(X_val).todense()

pred_val = search.predict(X_val)

print("Confusion matrix:")
print(confusion_matrix(y_val, pred_val))

print("\nF1 Score = {:.5f}".format(f1_score(y_val, pred_val, average='micro')))

print("\nClassification Report:")
print(classification_report(y_val, pred_val))

Confusion matrix:
[[224  33]
 [ 66 157]]

F1 Score = 0.79375

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.87      0.82       257
           1       0.83      0.70      0.76       223

    accuracy                           0.79       480
   macro avg       0.80      0.79      0.79       480
weighted avg       0.80      0.79      0.79       480



## Evaluation of Validation Set

In [181]:
# Print out the performance on the validation set
features_test = feature_processing_obj.transform(test['Sentence']).todense()
pred_test = search.predict(test['Sentence'])

y_test = test['Polarity']

print("Confusion matrix:")
print(confusion_matrix(y_test, pred_test))

print("\nF1 Score = {:.5f}".format(f1_score(y_test, pred_test, average="micro")))

print("\nClassification Report:")
print(classification_report(y_test, pred_test))

Confusion matrix:
[[237  50]
 [154 159]]

F1 Score = 0.66000

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.83      0.70       287
           1       0.76      0.51      0.61       313

    accuracy                           0.66       600
   macro avg       0.68      0.67      0.65       600
weighted avg       0.69      0.66      0.65       600



## Baseline Model - Retrained on Entire Training Set
We can see below that retraining the model yielded slightly different parameters, and an increased F1 score of 0.697 on the validation set as opposed to 0.66 previously.

In [182]:
search = search.fit(train['Sentence'], train['Polarity'])

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   40.3s
[Parallel(n_jobs=3)]: Done 144 out of 144 | elapsed:  5.6min finished


In [183]:
print("Best parameter (CV scy_train%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV scy_train0.739):
{'clf__n_estimators': 500, 'features__bow__cv__max_features': 1000, 'features__bow__cv__preprocessor': <function my_preprocess at 0x000001E3CB373E58>, 'features__bow__cv__use_idf': False, 'features__topics__cv__stop_words': None, 'features__topics__nmf__n_components': 75}


In [184]:
# The pipeline with the best performance
pipeline = search.best_estimator_
feature_processing_obj = pipeline.named_steps['features']

features_test = feature_processing_obj.transform(test['Sentence']).todense()
pred_test = search.predict(test['Sentence'])

y_test = test['Polarity']

print("Confusion matrix:")
print(confusion_matrix(y_test, pred_test))

print("\nF1 Score = {:.5f}".format(f1_score(y_test, pred_test, average="micro")))

print("\nClassification Report:")
print(classification_report(y_test, pred_test))

Confusion matrix:
[[235  52]
 [130 183]]

F1 Score = 0.69667

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.82      0.72       287
           1       0.78      0.58      0.67       313

    accuracy                           0.70       600
   macro avg       0.71      0.70      0.69       600
weighted avg       0.71      0.70      0.69       600



## Model Iteration 2 - Removal of Topics
We want to see if having topics actually helps, given that the validation set is from a different source, I hypothesize that it doesn't.

In [185]:
feature_processing =  FeatureUnion([ 
    ('bow', Pipeline([('cv', vectorizer), ])),
    ('length', FunctionTransformer(doc_length, validate=False)),
    ('words', FunctionTransformer(lexicon_count, validate=False)),
    ('punc_count', FunctionTransformer(punc_count, validate=False)),
    ('capital_count', FunctionTransformer(capital_count, validate=False)),  
    ('num_exclamation_marks', FunctionTransformer(num_exclamation_marks, validate=False)),  
    ('num_question_marks', FunctionTransformer(num_question_marks, validate=False)),  
    ('has_money', FunctionTransformer(has_money, validate=False)),
])

pipe = Pipeline([('features', feature_processing), ('clf', rf)])

param_grid = {
    'features__bow__cv__preprocessor': [None, my_preprocess],
    'features__bow__cv__max_features': [200, 500, 1000],
    'features__bow__cv__use_idf': [True, False],
    'clf__n_estimators': [100, 500],
}
    

search_no_topics = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3, scoring='f1_micro', return_train_score=True, verbose=2)

In [186]:
search_no_topics = search_no_topics.fit(train['Sentence'], train['Polarity'])

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   18.4s
[Parallel(n_jobs=3)]: Done  72 out of  72 | elapsed:  1.4min finished


We can see that the F1 Score went up to 0.70 without topics. So the model is better off without topics.

In [187]:
# The pipeline with the best performance
pipeline = search_no_topics.best_estimator_
feature_processing_obj = pipeline.named_steps['features']

features_test = feature_processing_obj.transform(test['Sentence']).todense()
pred_test = search_no_topics.predict(test['Sentence'])

y_test = test['Polarity']

print("Confusion matrix:")
print(confusion_matrix(y_test, pred_test))

print("\nF1 Score = {:.5f}".format(f1_score(y_test, pred_test, average="micro")))

print("\nClassification Report:")
print(classification_report(y_test, pred_test))

Confusion matrix:
[[239  48]
 [132 181]]

F1 Score = 0.70000

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.83      0.73       287
           1       0.79      0.58      0.67       313

    accuracy                           0.70       600
   macro avg       0.72      0.71      0.70       600
weighted avg       0.72      0.70      0.70       600



## Model Iteration 3 - Addition of Text Blob Sentiment Polarity Feature

In [188]:
feature_processing =  FeatureUnion([ 
    ('bow', Pipeline([('cv', vectorizer), ])),
    ('length', FunctionTransformer(doc_length, validate=False)),
    ('words', FunctionTransformer(lexicon_count, validate=False)),
    ('punc_count', FunctionTransformer(punc_count, validate=False)),
    ('capital_count', FunctionTransformer(capital_count, validate=False)),  
    ('num_exclamation_marks', FunctionTransformer(num_exclamation_marks, validate=False)),  
    ('num_question_marks', FunctionTransformer(num_question_marks, validate=False)),  
    ('has_money', FunctionTransformer(has_money, validate=False)),
    ('sent_polarity', FunctionTransformer(sent_polarity, validate=False)),
])

pipe = Pipeline([('features', feature_processing), ('clf', rf)])

param_grid = {
    'features__bow__cv__preprocessor': [None, my_preprocess],
    'features__bow__cv__max_features': [200, 500, 1000],
    'features__bow__cv__use_idf': [True, False],
    'clf__n_estimators': [100, 500],
}
    
search_sent = GridSearchCV(pipe, param_grid, cv=3, n_jobs=3, scoring='f1_micro', return_train_score=True, verbose=2)

In [189]:
search_sent = search_sent.fit(train['Sentence'], train['Polarity'])

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   33.8s
[Parallel(n_jobs=3)]: Done  72 out of  72 | elapsed:  1.7min finished


In [190]:
# Print out the results of hyperparmater tuning

def cv_results_to_df(cv_results):
    results = pd.DataFrame(list(cv_results['params']))
    results['mean_fit_time'] = cv_results['mean_fit_time']
    results['mean_score_time'] = cv_results['mean_score_time']
    results['mean_train_score'] = cv_results['mean_train_score']
    results['std_train_score'] = cv_results['std_train_score']
    results['mean_test_score'] = cv_results['mean_test_score']
    results['std_test_score'] = cv_results['std_test_score']
    results['rank_test_score'] = cv_results['rank_test_score']

    results = results.sort_values(['mean_test_score'], ascending=False)
    return results

results = cv_results_to_df(search_sent.cv_results_)
results

,clf__n_estimators,features__bow__cv__max_features,features__bow__cv__preprocessor,features__bow__cv__use_idf,mean_fit_time,mean_score_time,mean_train_score,std_train_score,mean_test_score,std_test_score,rank_test_score
18,500,500,<function my_preprocess at 0x000001E3CB373E58>,True,4.089843,0.458901,0.999375,0.00051,0.809583,0.016843,1
9,100,1000,None,False,1.588381,0.369888,0.999375,0.00051,0.809583,0.022646,1
19,500,500,<function my_preprocess at 0x000001E3CB373E58>,False,3.987691,0.450331,0.999375,0.00051,0.809167,0.014872,3
17,500,500,None,False,3.946029,0.414301,0.999375,0.00051,0.808333,0.014872,4
5,100,500,None,False,1.587208,0.362677,0.999375,0.00051,0.807917,0.011008,5
6,100,500,<function my_preprocess at 0x000001E3CB373E58>,True,1.750615,0.431381,0.999375,0.00051,0.807083,0.016499,6
10,100,1000,<function my_preprocess at 0x000001E3CB373E58>,True,1.655012,0.409862,0.999375,0.00051,0.806667,0.016019,7
22,500,1000,<function my_preprocess at 0x000001E3CB373E58>,True,4.200476,0.497941,0.999375,0.00051,0.805833,0.017510,8
21,500,1000,None,False,4.112860,0.493638,0.999375,0.00051,0.804583,0.014482,9
13,500,200,None,False,4.897429,0.582011,0.998958,0.00078,0.804583,0.014767,9


In [198]:
print("Best parameter (CV f1 score %0.3f):" % search_sent.best_score_)
print(search_sent.best_params_)

Best parameter (CV f1 score 0.810):
{'clf__n_estimators': 100, 'features__bow__cv__max_features': 1000, 'features__bow__cv__preprocessor': None, 'features__bow__cv__use_idf': False}


### Final Results on Validaiton Set
The Final F1 Score is 0.77 and the accuracy is also 0.77.

In [191]:
# The pipeline with the best performance
# Get the performance on the test set for the sentiment model
pipeline = search_sent.best_estimator_
feature_processing_obj = pipeline.named_steps['features']

features_test = feature_processing_obj.transform(test['Sentence']).todense()
pred_test = search_sent.predict(test['Sentence'])

y_test = test['Polarity']

print("Confusion matrix:")
print(confusion_matrix(y_test, pred_test))

print("\nF1 Score = {:.5f}".format(f1_score(y_test, pred_test, average="micro")))

print("\nClassification Report:")
print(classification_report(y_test, pred_test))

Confusion matrix:
[[248  39]
 [101 212]]

F1 Score = 0.76667

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.86      0.78       287
           1       0.84      0.68      0.75       313

    accuracy                           0.77       600
   macro avg       0.78      0.77      0.77       600
weighted avg       0.78      0.77      0.77       600



# Evaluation
## Feature Importance

In [192]:
# Because we are using a pipeline_best and a GridSearchCV, things are a bit complicated.
# I want to get references to the objects from the pipeline_best with the *best* hyperparameter settings,
# so that I can explore those objects (later). 

# The pipeline_best with the best performance
pipeline_best = search_sent.best_estimator_

# Get the feature processing pipeline_best, so I can use it later
feature_processing_obj = pipeline_best.named_steps['features']

# Find the vectorizer objects and the classifier objects
pipevect= dict(pipeline_best.named_steps['features'].transformer_list)
vectorizer_obj = pipevect.get('bow').named_steps['cv']
clf_obj = pipeline_best.named_steps['clf']

# Sanity check - what was vocabSize set to? Should match the output here.
len(vectorizer_obj.get_feature_names())

1000

In [197]:
stat_feature_names = [t[0] for t in pipeline_best.named_steps['features'].transformer_list if t[0] not in ['bow']]

feature_names = vectorizer_obj.get_feature_names() + stat_feature_names
len(feature_names)

feature_importances = None
if hasattr(clf_obj, 'feature_importances_'):
    feature_importances = clf_obj.feature_importances_

1008

In [194]:
features_train = feature_processing_obj.transform(X_train).todense()

N = features_train.shape[1]
ssum = np.zeros(N)
avg = np.zeros(N)
avg_pos = np.zeros(N)
avg_neg = np.zeros(N)
for i in range(N):
    ssum[i] = sum(features_train[:, i]).reshape(-1, 1)
    avg[i] = np.mean(features_train[:, i]).reshape(-1, 1)
    avg_pos[i] = np.mean(features_train[y_train==1, i]).reshape(-1, 1)
    avg_neg[i] = np.mean(features_train[y_train==0, i]).reshape(-1, 1)

#rf_new = search_sent.best_estimator_
imp = pd.DataFrame(data={'feature': feature_names, 'imp': feature_importances, 'sum': ssum, 'avg': avg, 'avg_pos': avg_pos, 'avg_neg': avg_neg})
imp = imp.sort_values(by='imp', ascending=False)
imp.head(20)
imp.tail(10)
#imp.to_csv('importances.csv', index=False)

,feature,imp,sum,avg,avg_pos,avg_neg
1007,sent_polarity,0.207102,308.695930,0.160779,0.423238,-0.103876
1000,length,0.025563,114560.000000,59.666667,57.650415,61.699791
1001,words,0.020817,20969.000000,10.921354,10.445021,11.401674
592,not,0.020122,51.439940,0.026792,0.007530,0.046215
406,great,0.017058,40.736042,0.021217,0.041235,0.001031
1003,capital_count,0.013469,3935.000000,2.049479,1.846473,2.254184
1002,punc_count,0.010572,3912.000000,2.037500,1.970954,2.104603
28,Great,0.010068,23.761461,0.012376,0.024649,0.000000
399,good,0.009465,47.568005,0.024775,0.037656,0.011786
469,is,0.009033,127.926627,0.066628,0.078844,0.054311


,feature,imp,sum,avg,avg_pos,avg_neg
822,they have,0.000009,1.705310,0.000888,0.000786,0.000991
1,20,0.000009,1.297826,0.000676,0.000529,0.000824
613,on and,0.000009,1.311835,0.000683,0.000428,0.000941
196,because they,0.000007,1.553688,0.000809,0.000538,0.001082
663,potato,0.000007,1.251312,0.000652,0.001082,0.000218
243,cell phone,0.000005,1.534416,0.000799,0.001055,0.000541
125,and friendly,0.000005,1.079867,0.000562,0.001120,0.000000
275,data,0.000005,0.742960,0.000387,0.000544,0.000228
858,to have,0.000003,1.521909,0.000793,0.001155,0.000427
563,minutes to,0.000003,0.836725,0.000436,0.000000,0.000875


## Investigating Incorrect Predictions Using Tree Interpreter
More can be read about tree interpreter here: https://blog.datadive.net/interpreting-random-forests/

In [195]:
X_test = test['Sentence']
y_test = test['Polarity']

In [196]:
from treeinterpreter import treeinterpreter as ti
features_test = feature_processing_obj.transform(test['Sentence']).todense()
prediction, bias, contributions = ti.predict(clf_obj, features_test)


for i in range(len(features_test)):
    if y_test.iloc[i] == pred_test[i]:
        continue
    print("Instance {}".format(i))
    X_test.iloc[i]
    print("Bias (trainset mean) {}".format(bias[i]))
    print("Truth {}".format(y_test.iloc[i]))
    print("Prediction {}".format(prediction[i, :]))
    print("Feature contributions:")
    con = pd.DataFrame(data={'feature': feature_names, 
                             'value': features_test[i].A1,
                             'negative contr': contributions[i][:, 0],
                             'positive contr': contributions[i][:, 1],
                             'abs contr': abs(contributions[i][:, 1])})

    con = con.sort_values(by="abs contr", ascending=False)
    con['positive cumulative'] = con['positive contr'].cumsum() + bias[i][1]
    con.head(30)
    print("-"*20) 

Instance 3


"It's a feel-good film and that's how I felt when I came out of the cinema!  "

Bias (trainset mean) [0.50557083 0.49442917]
Truth 1
Prediction [0.53 0.47]
Feature contributions:


,feature,value,negative contr,positive contr,abs contr,positive cumulative
399,good,0.267261,-0.122160,0.122160,0.122160,0.616589
1007,sent_polarity,0.000000,0.119441,-0.119441,0.119441,0.497147
1004,num_exclamation_marks,1.000000,-0.066050,0.066050,0.066050,0.563197
603,of,0.267261,0.023359,-0.023359,0.023359,0.539838
350,film,0.267261,-0.021663,0.021663,0.021663,0.561501
34,It,0.267261,0.021111,-0.021111,0.021111,0.540390
607,of the,0.267261,0.016714,-0.016714,0.016714,0.523676
592,not,0.000000,-0.014477,0.014477,0.014477,0.538153
946,when,0.267261,-0.011406,0.011406,0.011406,0.549560
780,that,0.267261,0.009257,-0.009257,0.009257,0.540303


--------------------
Instance 10


...

--------------------
Instance 591


'Fans of the genre will be in heaven.  '

Bias (trainset mean) [0.50557083 0.49442917]
Truth 1
Prediction [0.78 0.22]
Feature contributions:


,feature,value,negative contr,positive contr,abs contr,positive cumulative
1007,sent_polarity,0.000000,0.141957,-0.141957,0.141957,0.352472
603,of,0.408248,0.032449,-0.032449,0.032449,0.320024
592,not,0.000000,-0.017450,0.017450,0.017450,0.337474
1000,length,38.000000,-0.016853,0.016853,0.016853,0.354327
607,of the,0.408248,0.016562,-0.016562,0.016562,0.337765
1002,punc_count,1.000000,-0.012161,0.012161,0.012161,0.349926
1004,num_exclamation_marks,0.000000,0.010767,-0.010767,0.010767,0.339159
406,great,0.000000,0.010100,-0.010100,0.010100,0.329058
957,will,0.408248,-0.008857,0.008857,0.008857,0.337916
1003,capital_count,1.000000,-0.008703,0.008703,0.008703,0.346619


--------------------
